# Slater module about the slater's rule

##### Germain Salvato-Vallverdu [<germain.vallverdu@univ-pau.fr>](mailto:germain.vallverdu@univ-pau.fr)

* [Atomic orbitals](#Atomic-orbitals)
* [The Klechkowski class](#The-Klechkowski-class)
* [The ElectronicConf class](#The-ElectronicConf-class)
* [Example of Chromium exception](#Example-of-Chromium-exception)

In [1]:
import slater

In [2]:
print(slater.__doc__)


Slater Module
-------------

This module implement several classes in order to deal with atomic orbital,
electronic configuration and compute atomic orbital energies using the slater's
rule.

Important note: In the following, electronic sub-shell such as 2s, 3d ... are
called atomic orbital (AO) for simplicity. However, 2p, for example, is an
electronic sub-shell which contains 3 AO 2p_0, 2p_-1, 2p_1.

The implementation of this module is strongly inspired by the 
electronic_structure.core module of [pymatgen](http://pymatgen.org/) package.



### Atomic orbitals

#### Atomic orbitals type

The class ``AOType`` store the type of the atomic orbitals : s, p, d ...

In [3]:
print(slater.AOType.all_shells)

(s, p, d, f)


In [4]:
print(slater.AOType.s)

s


In [5]:
p_type = slater.AOType.from_string("p")
print(p_type)

p


In [6]:
f_type = slater.AOType.from_int(3)
print(f_type, "   l = ", f_type.l)

f    l =  3


#### The atomic orbital class

Using the ``AO`` class of the slater module.

In [7]:
AO_2s = slater.AO(n=2, aoType=slater.AOType.s, occ=1)

In [8]:
print(AO_2s)

2s


In [9]:
print("n = ", AO_2s.n, "\nl = ", AO_2s.l)

n =  2 
l =  0


In [10]:
print(AO_2s.name)

2s


In [11]:
print("degeneracy (2l + 1) = ", AO_2s.degeneracy)

degeneracy (2l + 1) =  1


An occupency can be set to the shell.

In [12]:
print(AO_2s.occ)

1


You can define the AO from a usual string.

In [13]:
OA_3d = slater.AO.from_string("3d")

In [14]:
print("OA : ", OA_3d.name, "\nn = ", OA_3d.n, "\nl = ", OA_3d.l, "\ndeg = ", OA_3d.degeneracy)

OA :  3d 
n =  3 
l =  2 
deg =  5


Other ways to define the AO :

In [15]:
OA_4p = slater.AO(4, "p")
print(OA_4p)

4p


In [16]:
OA_3s = slater.AO(3, 0)
print(OA_3s)

3s


### The Klechkowski class

This class simply implements the Klechlowski rule. You should not need to use it directly.

In [17]:
k = slater.Klechkowski()

In [18]:
print(k)

 1s
 2s 2p
 3s 3p
 4s 3d 4p
 5s 4d 5p
 6s 4f 5d 6p
 7s 5f 6d 7p


### The ElectronicConf class

This is the main part of the module.

#### Create the object

* You can start with the number of electrons :

In [19]:
Na = slater.ElectronicConf(nelec=11)

In [20]:
print(Na)

1s^2 2s^2 2p^6 3s^1


* Or you can give your own electronic configuration.
* Your own configuration might be wrong. Only the number of electrons by sub-shell is checked

In [21]:
Ca = slater.ElectronicConf.from_string("1s^2 2s^2 2p^6 3s^2 3p^6 4s^2")
print(Ca)

1s^2 2s^2 2p^6 3s^2 3p^6 4s^2


You can export the configuration in as a latex formula.

In [22]:
print(Na.toTex())

$\text{1s}^\text{2}\text{2s}^\text{2}\text{2p}^\text{6}\text{3s}^\text{1}$


You can print valence electrons :

In [23]:
print(Na.valence)

3s^1


#### Compute energies using slater's rule

In [24]:
data = Na.computeEnergy()

-----------------------------------
# AO   sigma     Z*    eps (eV)
-----------------------------------
  1s    0.31   10.69  -1554.38
  2s    4.15    6.85   -159.56
  2p    4.15    6.85   -159.56
  3s    8.80    2.20     -7.31
-----------------------------------


In [25]:
print(data[slater.AO.from_string("2p")])

(4.15, -159.55996125)


In [26]:
sigma, e = data[slater.AO.from_string("3s")]
print("sigma = ", sigma, "\ne = ", e, "eV")

sigma =  8.8 
e =  -7.314853333333327 eV


Energy of the configuration

In [27]:
print("E = ", Na.energy)

E =  -4392.561567733333


#### Play with ions

Simplest cases :

In [28]:
print("Na  :", Na)
print("q   :", Na.q, "    Z = ", Na.Z)
Na_p = Na.ionize(1)
print("Na + :", Na_p)
print("q   :", Na_p.q, "    Z = ", Na_p.Z)

Na  : 1s^2 2s^2 2p^6 3s^1
q   : 0     Z =  11
Na + : 1s^2 2s^2 2p^6
q   : 1     Z =  11


In [29]:
Cl = slater.ElectronicConf(nelec=17)
print("Cl  :", Cl)
Cl_m = Cl.ionize(-1)
print("Cl- :", Cl_m)

Cl  : 1s^2 2s^2 2p^6 3s^2 3p^5
Cl- : 1s^2 2s^2 2p^6 3s^2 3p^6


More complicated (4s / 3d inversion) : vanadium

In [30]:
V = slater.ElectronicConf(nelec=23)
print("V   :", V)
for i in [1, 2, 3]:
    ion = V.ionize(i)
    print("V{}+ :".format(ion.q), ion)

V   : 1s^2 2s^2 2p^6 3s^2 3p^6 4s^2 3d^3
V1+ : 1s^2 2s^2 2p^6 3s^2 3p^6 4s^1 3d^3
V2+ : 1s^2 2s^2 2p^6 3s^2 3p^6 3d^3
V3+ : 1s^2 2s^2 2p^6 3s^2 3p^6 3d^2


### Example of Chromium exception

Electronic configuration of chromium presents an exception. Here we compute the energy difference between the electronic configuration following the Klechkowski rule and the one with the exception.

In [31]:
Cr = slater.ElectronicConf(nelec=24)
print(Cr)

1s^2 2s^2 2p^6 3s^2 3p^6 4s^2 3d^4


In [32]:
Cr_exc = slater.ElectronicConf.from_string("1s^2 2s^2 2p^6 3s^2 3p^6 4s^1 3d^5")
print(Cr_exc)

1s^2 2s^2 2p^6 3s^2 3p^6 4s^1 3d^5


In [33]:
d = Cr.computeEnergy()

-----------------------------------
# AO   sigma     Z*    eps (eV)
-----------------------------------
  1s    0.31   23.69  -7633.66
  2s    4.15   19.85  -1339.87
  2p    4.15   19.85  -1339.87
  3s   11.25   12.75   -245.69
  3p   11.25   12.75   -245.69
  4s   20.55    3.45    -10.12
  3d   19.05    4.95    -37.03
-----------------------------------


In [34]:
d_exc = Cr_exc.computeEnergy()

-----------------------------------
# AO   sigma     Z*    eps (eV)
-----------------------------------
  1s    0.31   23.69  -7633.66
  2s    4.15   19.85  -1339.87
  2p    4.15   19.85  -1339.87
  3s   11.25   12.75   -245.69
  3p   11.25   12.75   -245.69
  4s   21.05    2.95     -7.40
  3d   19.40    4.60    -31.98
-----------------------------------


In [35]:
Cr.energy < Cr_exc.energy

True

Conclusion : Slater's rule  is not accurate enough to describe the chromium exception.